### Retrieve global coins blockchain data

In [162]:
import time, datetime
import requests
import pandas as pd
pd.set_option('display.precision', 10)

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockchain data:')
# print(df)
global_df

Global blockchain data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,613.000,3.1250000000,0.0033295067,SHA-256,2.6948384075,140.9461663948,0.0104047083,1.4665037500,0.5441898653
BTC,650.000,3.1695690500,1.0000000000,SHA-256,754.3764523085,132.9230769231,3.1695690500,421.3088706462,0.5584862430
DOGE,64.000,10000.0000000000,0.0000026550,Scrypt,0.0016329000,1350.0000000000,0.0265500811,35.8426094189,21950.2788562250
KAS,1.001,65.5729307474,0.0000011212,kHeavyHash,1.2557675347,86313.6863136863,0.0000735189,6.3456865043,5.0532334441
LTC,163.000,6.2500000000,0.0014190000,Scrypt,0.0020896573,530.0613496933,0.0088687500,4.7009815951,2249.6423520706
RVN,60.000,2500.0000000000,0.0000001483,KawPow,0.0000066832,1440.0000000000,0.0003706631,0.5337548242,79865.0341535240


### Retrieve NiceHash marketplace algorithms data

In [163]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,paying_btc_eh_day_$,daily_consumption_btc
SCRYPT,0.0001064779,17566.2997803389,"$1,673,131,909.63",1.8704225209
SHA256ASICBOOST,4.2627063688,0.5474501925,"$52,142.82",2.3336194221
KAWPOW,0.0000007267,75414.9502141395,"$7,183,024,384.41",0.0548050202
KHEAVYHASH,0.0465825693,4.8796510979,"$464,770.62",0.2273066856


### Calculate Nicehash max daily and max hourly consumption rate

In [ ]:
max_market_utilisation = 0.75

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,0.0000809232,1.4215211159,0.0592300465,"$5,641.47"
SHA256ASICBOOST,3.2396568403,1.7735507608,0.0738979484,"$7,038.53"
KAWPOW,0.0000005523,0.0416518154,0.0017354923,$165.30
KHEAVYHASH,0.0354027527,0.1727530810,0.0071980450,$685.59


### Calculate Nicehash hashrate to global hashrate ratio
Cap Nicehash hashrate to 15% of global hashrate

In [165]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_to_global_ratio_%'] = reachable_df['nh_to_global_ratio'] * 100
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']
reachable_df['nh_to_global_ratio_capped'] = reachable_df['nh_hashrate_eh_s_capped'] / reachable_df['nethash_eh_s']  

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_to_global_ratio_%,nh_hashrate_eh_s_capped,nh_to_global_ratio_capped
BCH,SHA-256,2.6948384075,3.2396568403,1.2021710954,120.2171095420,0.4042257611,0.1500000000
BTC,SHA-256,754.3764523085,3.2396568403,0.0042944830,0.4294482987,3.2396568403,0.0042944830
DOGE,Scrypt,0.0016329000,0.0000809232,0.0495579630,4.9557963041,0.0000809232,0.0495579630
KAS,kHeavyHash,1.2557675347,0.0354027527,0.0281921229,2.8192122915,0.0354027527,0.0281921229
LTC,Scrypt,0.0020896573,0.0000809232,0.0387255820,3.8725582044,0.0000809232,0.0387255820
RVN,KawPow,0.0000066832,0.0000005523,0.0826401990,8.2640199001,0.0000005523,0.0826401990


### Calculate burn rate needed for specific amount to be spent in the time period
```
B - burn rate
A - amount
P - price
D - duration

B = A / P * D
```

In [166]:
min_order_amount_btc = 0.0001 # BTC
order_duration_s = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['amount_btc'] = min_order_amount_btc
speed_df['duration_s'] = order_duration_s
speed_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
speed_df['speed_eh_s'] = speed_df.amount_btc / (speed_df.paying_btc_eh_s * speed_df.duration_s)
speed_df

,amount_btc,duration_s,paying_btc_eh_s,speed_eh_s
SCRYPT,0.0001,3600,0.2033136549,0.0000001366
SHA256ASICBOOST,0.0001,3600,0.0000063362,0.0043839605
KAWPOW,0.0001,3600,0.8728582201,0.0000000318
KHEAVYHASH,0.0001,3600,0.0000564774,0.0004918384


### Calculate amount needed to buy speed for the time period
```
B - burn rate
A - amount
P - price
D - duration

A = B * P * D 
```

In [167]:
speed_eh_s = 1 # EH / s
order_duration_s = 3600  # 1 hour

amount_df = pd.DataFrame(index=nh_market_df.index)
amount_df['speed_eh_s'] = speed_eh_s
amount_df['duration_s'] = order_duration_s
amount_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
amount_df['amount_btc'] = amount_df.speed_eh_s * amount_df.paying_btc_eh_s * amount_df.duration_s
amount_df

,speed_eh_s,duration_s,paying_btc_eh_s,amount_btc
SCRYPT,1,3600,0.2033136549,731.9291575141
SHA256ASICBOOST,1,3600,0.0000063362,0.0228104247
KAWPOW,1,3600,0.8728582201,3142.2895922558
KHEAVYHASH,1,3600,0.0000564774,0.2033187957


### Calculate RTP for the coin on the NiceHash market

In [168]:
import pandas as pd
import numpy as np

max_allowed_coin_market_hp_share = 0.15
max_nicehash_market_utilisation = 0.76
speed_eh_s = 1
order_duration_s = 3600
order_amount_btc = 1

rtp_df = pd.DataFrame()
# rtp_df.loc['BCH', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BCH', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['BTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['DOGE', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['DOGE', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['LTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['LTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['KAS', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KHEAVYHASH', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['KAS', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['RVN', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KAWPOW', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['RVN', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df['nh_to_global_ratio'] = rtp_df['nh_hashrate_eh_s'] / global_df['nethash_eh_s']
# rtp_df['max_global_btc_reward_h'] = rtp_df.nh_to_global_ratio * global_df['daily_emmisions_btc'] / 24
# rtp_df['max_global_btc_reward_h'] = global_df['daily_emmisions_btc'] / 24
rtp_df['global_price_btc_eh_s'] = global_df['index_price_btc_eh_day'] / 86400
rtp_df.loc['BCH', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['BTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['DOGE', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
# rtp_df.loc['LTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
# rtp_df.loc['KAS', 'nh_price_btc_eh_s'] = nh_market_df.loc['KHEAVYHASH', 'paying_btc_eh_day'] / 86400
# rtp_df.loc['RVN', 'nh_price_btc_eh_s'] = nh_market_df.loc['KAWPOW', 'paying_btc_eh_day'] / 86400
rtp_df['global_1eh_1h_duration_reward_btc'] = rtp_df.global_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['nh_1eh_1h_duration_expense_btc'] = rtp_df.nh_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['rtp_%'] =  rtp_df.global_1eh_1h_duration_reward_btc / rtp_df.nh_1eh_1h_duration_expense_btc * 100
rtp_df['allocation_%'] = rtp_df['rtp_%'] / rtp_df['rtp_%'].sum() * 100

# timestamp = datetime.datetime.fromtimestamp(time.time()).isoformat()
# rtp_df.to_csv(f'RTP - {timestamp}.csv')

rtp_df


,global_price_btc_eh_s,nh_price_btc_eh_s,global_1eh_1h_duration_reward_btc,nh_1eh_1h_duration_expense_btc,rtp_%,allocation_%
BCH,0.0000062985,0.0000063362,0.0226745777,0.0228104247,99.4044522764,30.4569311911
BTC,0.0000064640,0.0000063362,0.0232702601,0.0228104247,102.0159003800,31.2570633117
DOGE,0.2540541534,0.2033136549,914.5949523427,731.9291575141,124.9567588548,38.2860054972
KAS,0.0000584865,NaN,0.2105513935,NaN,NaN,NaN
LTC,0.0260375272,NaN,93.7350980029,NaN,NaN,NaN
RVN,0.9243638212,NaN,3327.7097563968,NaN,NaN,NaN
